In [1]:
import random
from collections import deque

weather_states = ["rain", "snow", "clear", "humid"]

GOAL_THRESHOLD = 50
INITIAL_MOISTURE = 20

# ----------------------------------------
# Temperature Selection
# ----------------------------------------

def select_initial_temperature(weather):
    if weather == "rain":
        return "mild"
    elif weather == "snow":
        return "cold"
    elif weather == "humid":
        return "hot"
    elif weather == "clear":
        return random.choice(["hot", "warm", "mild"])

# ----------------------------------------
# Temperature Progression
# ----------------------------------------

def temperature_progression(initial_temp):
    if initial_temp == "hot":
        return ["hot", "warm", "mild"]
    elif initial_temp == "warm":
        return ["warm", "mild"]
    elif initial_temp == "mild":
        return ["mild"]
    else:
        return [initial_temp]

# ----------------------------------------
# Evaporation Model
# ----------------------------------------

def evaporation_rate(temp, weather):
    if weather == "humid":
        return 25
    if temp == "hot":
        return 35
    elif temp == "warm":
        return 20
    elif temp == "mild":
        return 5
    else:
        return 0

# ----------------------------------------
# Irrigation Gain
# ----------------------------------------

def irrigation_gain(minutes):
    if minutes == 10:
        return 25
    elif minutes == 20:
        return 45
    else:
        return 0

# ----------------------------------------
# Deliberate Agent with BFS Search
# ----------------------------------------

class DeliberateSprinklerAgent:

    def __init__(self, weather, initial_temp):
        self.weather = weather
        self.temp_cycles = temperature_progression(initial_temp)

    def transition(self, state, action):

        cycle_index, moisture = state
        temp = self.temp_cycles[cycle_index]

        # Irrigation effect
        moisture += irrigation_gain(action)
        moisture = min(moisture, 100)

        # Evaporation between cycles
        if cycle_index < len(self.temp_cycles) - 1:
            moisture -= evaporation_rate(temp, self.weather)
            moisture = max(moisture, 0)

        return (cycle_index + 1, moisture)

    def goal_test(self, state):
        cycle_index, moisture = state
        return cycle_index == len(self.temp_cycles) and moisture >= GOAL_THRESHOLD

    def generate_all_plans(self):

        if self.weather in ["rain", "snow"]:
            return []

        initial_state = (0, INITIAL_MOISTURE)
        queue = deque()
        queue.append((initial_state, []))

        all_results = []

        while queue:
            state, path = queue.popleft()
            cycle_index, moisture = state

            if cycle_index == len(self.temp_cycles):
                all_results.append((path, moisture, sum(path)))
                continue

            for action in [0, 10, 20]:
                new_state = self.transition(state, action)
                queue.append((new_state, path + [action]))

        return all_results

    def select_best_plan(self, all_plans):

        best_plan = None
        best_cost = float('inf')

        for plan, final_moisture, cost in all_plans:
            if final_moisture >= GOAL_THRESHOLD and cost < best_cost:
                best_cost = cost
                best_plan = plan

        return best_plan, best_cost

# ----------------------------------------
# Simulation Runner
# ----------------------------------------

def run_simulation():

    print("\nEnter weather for 5 days (rain/snow/clear/humid)\n")

    daily_inputs = []
    for i in range(5):
        w = input(f"Day {i+1} Weather: ").lower()
        while w not in weather_states:
            print("Invalid input.")
            w = input(f"Day {i+1} Weather: ").lower()
        daily_inputs.append(w)

    print("\n========== DELIBERATE AGENT WITH EVAPORATION SPEED ==========\n")

    for day in range(5):

        weather = daily_inputs[day]
        initial_temp = select_initial_temperature(weather)

        print(f"=========== DAY {day+1} ===========")
        print(f"Weather        : {weather}")
        print(f"Initial Temp   : {initial_temp}")
        print(f"Initial Moisture: {INITIAL_MOISTURE}\n")

        if weather in ["rain", "snow"]:
            print("Rain/Snow detected → Soil remains wet. No irrigation needed.\n")
            print("---------------------------------------------\n")
            continue

        agent = DeliberateSprinklerAgent(weather, initial_temp)

        all_plans = agent.generate_all_plans()

        print("All Plans Explored:\n")
        for plan, final_moisture, cost in all_plans:
            print(f"Plan: {plan} | Final Moisture: {final_moisture} | Cost: {cost}")

        best_plan, best_cost = agent.select_best_plan(all_plans)

        print("\nSelected Optimal Plan:")
        print(f"Best Plan: {best_plan}")
        print(f"Minimum Irrigation Cost: {best_cost}")
        print("---------------------------------------------\n")


run_simulation()


Enter weather for 5 days (rain/snow/clear/humid)



Day 1 Weather:  snow
Day 2 Weather:  clear
Day 3 Weather:  clear
Day 4 Weather:  humid
Day 5 Weather:  rain



========== DELIBERATE AGENT WITH EVAPORATION SPEED ==========

=========== DAY 1 ===========
Weather        : snow
Initial Temp   : cold
Initial Moisture: 20

Rain/Snow detected → Soil remains wet. No irrigation needed.

---------------------------------------------

=========== DAY 2 ===========
Weather        : clear
Initial Temp   : warm
Initial Moisture: 20

All Plans Explored:

Plan: [0, 0] | Final Moisture: 0 | Cost: 0
Plan: [0, 10] | Final Moisture: 25 | Cost: 10
Plan: [0, 20] | Final Moisture: 45 | Cost: 20
Plan: [10, 0] | Final Moisture: 25 | Cost: 10
Plan: [10, 10] | Final Moisture: 50 | Cost: 20
Plan: [10, 20] | Final Moisture: 70 | Cost: 30
Plan: [20, 0] | Final Moisture: 45 | Cost: 20
Plan: [20, 10] | Final Moisture: 70 | Cost: 30
Plan: [20, 20] | Final Moisture: 90 | Cost: 40

Selected Optimal Plan:
Best Plan: [10, 10]
Minimum Irrigation Cost: 20
---------------------------------------------

=========== DAY 3 ===========
Weather        : clear
Initial Temp   : mild
Init